Using GRPO to finetune the chat gpt2 model

# Imports

In [13]:
from importlib.metadata import version
import torch, tiktoken, time, os, tensorflow
import torch.optim as optim
import numpy as np
import argparse
from tqdm import tqdm
from torch.utils.data import Dataset, DataLoader
import zipfile
from pathlib import Path
import pandas as pd
from torch.nn import Module # For type hinting

pkgs = ["numpy", 
        "tiktoken", 
        "torch",
        # "tensorflow", # For OpenAI's pretrained weights
        "pandas"
       ]
for p in pkgs:
    print(f"{p} version: {version(p)}")

numpy version: 1.23.5
tiktoken version: 0.9.0
torch version: 2.5.1
pandas version: 2.3.1


In [2]:
!python --version

Python 3.10.8


In [2]:
os.cpu_count()

24

In [ ]:
from utils.previous_chapters import generate_text_simple, text_to_token_ids, token_ids_to_text,GPTModel, create_dataloader_v1, load_weights_into_gpt
# Relative import from the gpt_download.py contained in this folder
from utils.gpt_download import download_and_load_gpt2

## Dataset Class

In [8]:
def prepare_datasets(data_file_path, sep="\t", header=None, column_names=["Label", "Text"], train_frac=0.7, validation_frac=0.15, store_directory="./"):
    df = pd.read_csv(data_file_path, sep=sep, header=header, names=column_names)

    # Count the instances of "spam"
    num_spam = df[df["Label"] == "spam"].shape[0]
    
    # Randomly sample "ham" instances to match the number of "spam" instances
    ham_subset = df[df["Label"] == "ham"].sample(num_spam, random_state=123)
    
    # Combine ham "subset" with "spam"
    balanced_df = pd.concat([ham_subset, df[df["Label"] == "spam"]])

    balanced_df["Label"] = balanced_df["Label"].map({"ham": 0, "spam": 1})

    # Shuffle the entire DataFrame
    balanced_df = balanced_df.sample(frac=1, random_state=123).reset_index(drop=True)

    # Calculate split indices
    train_end = int(len(balanced_df) * train_frac)
    validation_end = train_end + int(len(balanced_df) * validation_frac)

    # Split the DataFrame
    train_df = balanced_df[:train_end]
    validation_df = balanced_df[train_end:validation_end]
    test_df = balanced_df[validation_end:]

    train_df.to_csv(store_directory+"/train.csv", index=None)
    validation_df.to_csv(store_directory+"/validation.csv", index=None)
    test_df.to_csv(store_directory+"/test.csv", index=None)

In [9]:
class SpamDataset(Dataset):
    def __init__(self, csv_file, tokenizer, max_length=None, pad_token_id=50256):
        try:
            self.data = pd.read_csv(csv_file)
        except FileNotFoundError:
            raise FileNotFoundError(f"File not found: {csv_file}")

        # Pre-tokenize texts
        self.encoded_texts = [
            tokenizer.encode(text) for text in self.data["Text"]    # For each row in the text section of the pandas data frame tokenize the text string(sentence); creates list of token IDs for each example/item of the text data
        ]

        if max_length is None:
            self.max_length = self._longest_encoded_length()
        else:
            self.max_length = max_length
            # Truncate sequences if they are longer than max_length
            self.encoded_texts = [
                encoded_text[:self.max_length]
                for encoded_text in self.encoded_texts
            ]

        # Pad sequences to the longest sequence
        self.encoded_texts = [
            encoded_text + [pad_token_id] * (self.max_length - len(encoded_text))
            for encoded_text in self.encoded_texts
        ]

    def __getitem__(self, index):
        encoded = self.encoded_texts[index]
        label = self.data.iloc[index]["Label"]
        return (
            torch.tensor(encoded, dtype=torch.long),
            torch.tensor(label, dtype=torch.long)
        )

    def __len__(self):
        return len(self.data)

    def _longest_encoded_length(self):
        max_length = 0
        for encoded_text in self.encoded_texts:
            encoded_length = len(encoded_text)
            if encoded_length > max_length:
                max_length = encoded_length
        return max_length

## Building Policies

In [10]:
def build_new_policy(base_config, chosen_model="gpt2-small (124M)", num_classes = 2) -> GPTModel:
    """Build and load in the GPT2 model. Swap out the Head layer, and freeze up to the last Transformer module for transfer learning."""
    model_configs = {
        "gpt2-small (124M)": {"emb_dim": 768, "n_layers": 12, "n_heads": 12},
        "gpt2-medium (355M)": {"emb_dim": 1024, "n_layers": 24, "n_heads": 16},
        "gpt2-large (774M)": {"emb_dim": 1280, "n_layers": 36, "n_heads": 20},
        "gpt2-xl (1558M)": {"emb_dim": 1600, "n_layers": 48, "n_heads": 25},
    }

    base_config.update(model_configs[chosen_model]) # add the emb_dim, n_layers, and n_heads to the config

    model_size = chosen_model.split(" ")[-1].lstrip("(").rstrip(")")
    allowed_sizes = ("124M", "355M", "774M", "1558M")
    if model_size not in allowed_sizes:
        raise ValueError(f"Model size not in {allowed_sizes}")
    settings, params = download_and_load_gpt2(model_size=model_size, models_dir="gpt2")

    model = GPTModel(base_config)

    load_weights_into_gpt(model, params)
    
    for param in model.parameters(): # freeze model parameters
        param.requires_grad = False 

    # Unfreeze the last transformer block
    for param in model.trf_blocks[-1].parameters():
        param.requires_grad = True

    # Unfreeze the final layer normalizing layer
    for param in model.final_norm.parameters():
        param.requires_grad = True

    model.out_head = torch.nn.Linear(in_features=base_config["emb_dim"], out_features=num_classes) # reconfigure the output layer
    return model

In [7]:
def build_old_policy(base_config, chosen_model="gpt2-small (124M)", num_classes = 2) -> GPTModel:
    """Construct just the model without loading the weights"""
    model_configs = {
        "gpt2-small (124M)": {"emb_dim": 768, "n_layers": 12, "n_heads": 12},
        "gpt2-medium (355M)": {"emb_dim": 1024, "n_layers": 24, "n_heads": 16},
        "gpt2-large (774M)": {"emb_dim": 1280, "n_layers": 36, "n_heads": 20},
        "gpt2-xl (1558M)": {"emb_dim": 1600, "n_layers": 48, "n_heads": 25},
    }

    base_config.update(model_configs[chosen_model]) # add the emb_dim, n_layers, and n_heads to the config

    model_size = chosen_model.split(" ")[-1].lstrip("(").rstrip(")")
    allowed_sizes = ("124M", "355M", "774M", "1558M")
    if model_size not in allowed_sizes:
        raise ValueError(f"Model size not in {allowed_sizes}")
    model = GPTModel(base_config)

    model.out_head = torch.nn.Linear(in_features=base_config["emb_dim"], out_features=num_classes) # reconfigure the output layer
    return model

## Utility Functions

In [11]:
def calculate_discounted_rewards(predictions, batch_labels) -> torch.tensor:
    """For this implementation, use non-discounted rewards"""
    disc_rewards = (predictions == batch_labels).float()    # Simple comparison to evaluate rewards for each example; output a tensor of floats
    return disc_rewards

In [ ]:
def log_epoch_stats(epoch, epoch_limit, total_loss, ratio, entropy) -> None:
    print(f"=====================  [Epoch ({epoch})]  =====================")
    print("Last k_epoch stats:")
    print(f"Loss: {total_loss:.7f} | Ratio: {ratio:.7f} | Entropy Term: {entropy:.7f}")
    print(f"===========================================================")


In [ ]:
def evaluate_policy(Policy: Module, dataloader: DataLoader, current_epoch: int = None, max_epochs: int=None, device: str = 'cpu') -> float:
    """
    Evaluates the policy model (greedy version) on a given dataset.

    Returns:
        accuracy (float): The calculated accuracy
    """
    Policy.eval()   # Turn off dropout layers and prevent grad tracking

    # Dataset check before continuing
    if len(dataloader.dataset) == 0: # Check the underlying dataset size
        print(f"Warning: Evaluation dataset is empty. Skipping accuracy calculation.")
        return float('nan')
    
    accuracy, num_correct, num_of_samples= 0.0, 0.0, 0.0

    Softmax_lyr = torch.nn.Softmax(dim=-1)
    
    with torch.no_grad():
        for batch_inputs, batch_labels in dataloader:
            batch_inputs, batch_labels = batch_inputs.to(device), batch_labels.to(device) # move the training data to the target device

            logits = Policy(batch_inputs)[:,-1,:]   # Get logits from model and only focus on the last iterations of each sample
            # print(old_logits)
            
            classification_probabilities = Softmax_lyr(logits)
            class_predictions = torch.argmax(classification_probabilities, dim=-1).flatten()
            num_of_samples += batch_labels.size(0)
            num_correct += sum((class_predictions == batch_labels).float()).item()
        accuracy = num_correct/num_of_samples
        if current_epoch and max_epochs:   # If the function was called in the training loop
            print(f"===================  [Epoch ({current_epoch}/{max_epochs})]  ===================")
            print(f"Entire Validation Dataset Accuracy: {accuracy:.4f}| {num_correct} / {num_of_samples} samples")
            print(f"====================================================")

        else:   # If the function was called outside of the training loop
            print(f"===============================================")
            print(f"Entire Dataset Accuracy: {accuracy:.4f} | {num_correct} / {num_of_samples} samples")
            print(f"=====================================================")

            
    Policy.train()  # set back to training mode 
    return accuracy

## Training Loop

In [106]:
def grpo_train(model_config, train_dataloader, validation_dataloader, gpt_size="gpt2-small (124M)", epochs=50, learning_rate=0.0001, batch_size=64, gamma=0.99, k_epochs=64, epsilon=0.2, beta_kl=0.01, max_grad_norm=0.5, entropy_coeff=0.01, log_iterations=10, eval_iterations=10, device="cpu", num_envs=None) -> Module:
    print(f"Training Policy on {device} with {epochs} main epochs, {k_epochs} inner epochs, {learning_rate} learning rate, batch size={batch_size}, KL beta={beta_kl}.")
    print(f"Using gpt2 size: '{gpt_size}', logging every {log_iterations} epoch iterations, evaluating every {eval_iterations} epoch iterations.")


    Policy_New = build_new_policy(model_config, chosen_model=gpt_size, num_classes=2).to(device)   # STEP 3 || 
    Policy_New.train()
    # Policy_New = torch.compile(Policy_New) # to reap efficiency benefits ; Not Working - triton dependency!!

    optimizer = optim.Adam(params=Policy_New.parameters(), lr=learning_rate)

    Policy_Old = build_old_policy(model_config, chosen_model=gpt_size, num_classes=2).to(device)
    Policy_Old.eval()
    # Policy_Old = torch.compile(Policy_Old)

    for epoch in tqdm(iterable=range(epochs), desc=f">>>>>>>>>>>>>>>>>>>>>\nMain Epoch (Outer Loop)", leave=True):     # STEP 4 || 
        # STEP 5 || Sample a batch D_b from D --> OMITTED 
        # STEP 6 || Update the old policy model PI old <- PI new
        Policy_Old.load_state_dict(Policy_New.state_dict())
        # print("loaded Policy Old Weights")
        # --- STEP 7 Collect a Batch of Experiences Using the Old Policy---
        # for batch_inputs, batch_labels in train_dataloader:
        #     pass
        batch_inputs, batch_labels = next(iter(train_dataloader))
        # print(f"batch_inputs shape: {batch_inputs.shape}")
        # print(f"batch_labels shape: {batch_labels.shape}")

        batch_inputs, batch_labels = batch_inputs.to(device), batch_labels.to(device) # move the training data to the target device
        
        with torch.no_grad():
            old_logits = Policy_Old(batch_inputs)[:,-1,:]   # Get logits from model and only focus on the last iterations of each sample
            # print(old_logits)
            # print(f"old_logits shape: {old_logits.shape}")

            old_dist = torch.distributions.Categorical(logits=old_logits) # Create a distribution to sample from
            old_predictions = old_dist.sample() # Tensor of shape (batch_size,) ; list of predictions
            print(f"old_predictions: \n{old_predictions[:10]}")
            print(f"batch_labels True Values: \n{batch_labels[:10]}")

            old_log_probs = old_dist.log_prob(old_predictions)

        # STEP 8 || Calculate "Discounted" Rewards for completed trajectories
        discounted_rewards = calculate_discounted_rewards(old_predictions, batch_labels)    # Tensor with "discounted" rewards per each sample in batch
        # print("Calculated discounted returns")
        print(f"discounted_rewards: \n{discounted_rewards[:10]} Shape: {discounted_rewards.shape}")

        # STEP 9 || Calculate the Advantage for each Trajectory using normalization
        all_advantages_tensor = (discounted_rewards - discounted_rewards.mean()) / (discounted_rewards.std() + 1e-8)
        print(f"all_advantages_tensor: \n{all_advantages_tensor[:10]}")


        Policy_New.train()  # Prepare NN for updates

        # --- STEP 10 || GRPO Optimization ---
        for k_epoch in tqdm(iterable=range(k_epochs), desc=f"Epoch {epoch+1}/{epochs} (Inner K-Epochs)", leave=True):
            print(f"===========================  [({k_epoch+1}/{k_epochs})]  ==========================\n")
            optimizer.zero_grad()   # Flush out all the accumulated gradients for the weights of the model-under-training

            new_logits = Policy_New(batch_inputs)[:,-1,:]   # Get logits from model and only focus on the last iterations of each sample
            new_dist = torch.distributions.Categorical(logits=new_logits)
            new_log_probs = new_dist.log_prob(old_predictions)  # Get the log probability of choosing the same action that the old policy took using the new distribution
            entropy = new_dist.entropy().mean() # Calculate entropy for regularization
            print(f"Entropy of this k_epoch: {entropy}")
            R1_ratio = torch.exp(new_log_probs - old_log_probs) # Exponent trick
            # print(f"R1_ratio: \n{R1_ratio[:10]}")

            unclipped_surrogate = R1_ratio * all_advantages_tensor
            clipped_surrogate = torch.clamp(input=R1_ratio, min=1.0-epsilon, max=1.0+epsilon) * all_advantages_tensor
            # print(f"unclipped_surrogate: \n{unclipped_surrogate[:10]}\nclipped_surrogate: \n{clipped_surrogate[:10]}")
            policy_loss = -torch.min(unclipped_surrogate, clipped_surrogate).mean()

            print(f"Average policy_loss of this k_epoch: {policy_loss}")

            # --- KL Divergence Calculation ---
            # Create distributions for old policies using the trajectory states
            # with torch.no_grad():
            #     old_logits = Policy_Old(all_states_tensor)
            # old_dist = torch.distributions.Categorical(logits=old_logits)
            # INSTEAD, just reusing the calculated logits from STEP #7

            # Calculate KL divergence per sample, then take the mean over the batch
            kl_div_per_sample = torch.distributions.kl.kl_divergence(p=new_dist, q=old_dist)
            # print(f"KL Divergence between new and old policy: \n{kl_div_per_sample[:10]}")
            kl_loss = kl_div_per_sample.mean() # Mean over the batch
            print(f"KL Divergence Average Loss: {kl_loss}")

            # Total Loss for GRPO
            total_loss = policy_loss + beta_kl * kl_loss - entropy_coeff * entropy
            print(f"Total Loss of this k_epoch: {total_loss}\n")
            # STEP 11 || Policy Updates
            total_loss.backward()
            torch.nn.utils.clip_grad_norm_(Policy_New.parameters(), max_grad_norm)
            optimizer.step()    # Update policy parameters using gradient ascent
                

        # --- Logging and Evaluation ---
        if (epoch + 1) % log_iterations == 0:
            log_epoch_stats(epoch=epoch+1, epoch_limit=epochs, total_loss=total_loss.item(), ratio=R1_ratio.mean().item(), entropy=entropy)

        if (epoch + 1) % eval_iterations == 0:
            accuracy = evaluate_policy(Policy_New, validation_dataloader, current_epoch=epoch+1, max_epochs=epochs, device=device)

    Policy_New.eval()   # Change to eval mode for evaluation after training is complete

    print("Training complete.")
    return Policy_New # Return the trained policy

## Main Loop

In [17]:
BASE_CONFIG = {
    "vocab_size": 50257,     # Vocabulary size
    "context_length": 1024,  # Context length
    "drop_rate": 0.1,        # Dropout rate
    "qkv_bias": True         # Query-key-value bias
}

In [108]:
# Pretend that the argument parser will pass these arguments to the main function
args = {
    "epochs":32,
    "learning_rate":0.0003,
    "dataloader_batch_size":64,
    "dataloader_pin_memory": True,  
    "dataloader_num_workers": 0,    # Problem if I change this; slow for windows; try to modify within .py script
    "batch_size":None, # Not needed in this build/project
    "gpt_size":'gpt2-small (124M)',
    "k_epochs":64,       # GRPO Inner-loop
    "epsilon":0.2,
    "beta_kl":0.01,
    "entropy_coeff":0.05,   # 
    "log_iterations":1,     # Log GRPO stats
    "eval_iterations":1,    # Run model through evaluation at every "x" epochs
    "gamma":None,   # Discounted Rewards
    "device":'cuda',
    "num_envs":None,
    "save_model":True,
    "model_output_path":'models/first.pt'
}

In [19]:
prepare_datasets(data_file_path="./sms_spam_collection/SMSSpamCollection.tsv", store_directory="./sms_spam_collection/data_splits")

In [20]:
tokenizer = tiktoken.get_encoding("gpt2")

In [21]:
train_dataset = SpamDataset(csv_file="./sms_spam_collection/data_splits/train.csv", tokenizer=tokenizer)
# test_dataset = SpamDataset(csv_file="./sms_spam_collection/data_splits/test.csv", tokenizer=tokenizer)
validation_dataset = SpamDataset(csv_file="./sms_spam_collection/data_splits/validation.csv", tokenizer=tokenizer)

In [22]:
train_dataloader = DataLoader(dataset=train_dataset, batch_size=args["dataloader_batch_size"], num_workers=args["dataloader_num_workers"], pin_memory=args["dataloader_pin_memory"], drop_last=True)
# test_dataloader = DataLoader(dataset=test_dataset, batch_size=args["dataloader_batch_size"], num_workers=args["dataloader_num_workers"], pin_memory=args["dataloader_pin_memory"], drop_last=True)
validation_dataloader = DataLoader(dataset=validation_dataset, batch_size=args["dataloader_batch_size"], num_workers=args["dataloader_num_workers"], pin_memory=args["dataloader_pin_memory"], drop_last=True)

In [23]:
print(len(train_dataloader))
print(len(validation_dataloader))


16
3


In [24]:
for batch_inputs, batch_labels in validation_dataloader:
    print(f"{batch_labels.size(0)}")
    break

64


In [109]:
# function call
trained_policy = grpo_train(
        model_config=BASE_CONFIG,
        train_dataloader=train_dataloader,
        validation_dataloader=validation_dataloader,
        gpt_size=args["gpt_size"],
        epochs=args["epochs"],
        eval_iterations=args["eval_iterations"],
        learning_rate=args["learning_rate"],
        batch_size=args["batch_size"], # Significantly larger batch size recommended for stability
        k_epochs=args["k_epochs"],
        epsilon=args["epsilon"],
        beta_kl=args["beta_kl"],
        entropy_coeff=args["entropy_coeff"],
        log_iterations=args["log_iterations"],
        gamma=args["gamma"],
        device=args["device"],
        num_envs=args["num_envs"]
    )

Training Policy on cuda with 32 main epochs, 64 inner epochs, 0.0003 learning rate, batch size=None, KL beta=0.01.
Using gpt2 size: 'gpt2-small (124M)', logging every 1 epoch iterations, evaluating every 1 epoch iterations.
File already exists and is up-to-date: gpt2\124M\checkpoint
File already exists and is up-to-date: gpt2\124M\encoder.json
File already exists and is up-to-date: gpt2\124M\hparams.json
File already exists and is up-to-date: gpt2\124M\model.ckpt.data-00000-of-00001
File already exists and is up-to-date: gpt2\124M\model.ckpt.index
File already exists and is up-to-date: gpt2\124M\model.ckpt.meta
File already exists and is up-to-date: gpt2\124M\vocab.bpe


>>>>>>>>>>>>>>>>>>>>>
Main Epoch (Outer Loop):   0%|          | 0/32 [00:00<?, ?it/s]

old_predictions: 
tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1], device='cuda:0')
batch_labels True Values: 
tensor([0, 0, 1, 1, 1, 0, 0, 1, 0, 0], device='cuda:0')
discounted_rewards: 
tensor([0., 0., 1., 1., 1., 0., 0., 1., 0., 0.], device='cuda:0') Shape: torch.Size([64])
all_advantages_tensor: 
tensor([-0.9922, -0.9922,  0.9922,  0.9922,  0.9922, -0.9922, -0.9922,  0.9922,
        -0.9922, -0.9922], device='cuda:0')


===========================  [(1/64)]  ==========================

Entropy of this k_epoch: 0.03249852731823921
Average policy_loss of this k_epoch: -0.00024547427892684937
KL Divergence Average Loss: 0.012347672134637833
Total Loss of this k_epoch: -0.0017469238955527544

===========================  [(2/64)]  ==========================

Entropy of this k_epoch: 0.08917655050754547
Average policy_loss of this k_epoch: 0.007896699011325836
KL Divergence Average Loss: 0.0696031004190445
Total Loss of this k_epoch: 0.004133902955800295

===========================  [(3/64)]  ==========================



Entropy of this k_epoch: 0.14105895161628723
Average policy_loss of this k_epoch: -0.015257537364959717
KL Divergence Average Loss: 0.19067054986953735
Total Loss of this k_epoch: -0.020403780043125153

===========================  [(4/64)]  ==========================

Entropy of this k_epoch: 0.1654437929391861
Average policy_loss of this k_epoch: 0.012618497014045715
KL Divergence Average Loss: 0.18602150678634644
Total Loss of this k_epoch: 0.006206522695720196

===========================  [(5/64)]  ==========================

Entropy of this k_epoch: 0.20468920469284058
Average policy_loss of this k_epoch: 0.007583394646644592
KL Divergence Average Loss: 0.23945797979831696
Total Loss of this k_epoch: -0.0002564862370491028

===========================  [(6/64)]  ==========================



...

Entropy of this k_epoch: 0.10698112845420837
Average policy_loss of this k_epoch: 0.002619341015815735
KL Divergence Average Loss: 0.017076538875699043
Total Loss of this k_epoch: -0.0025589498691260815

===========================  [(61/64)]  ==========================

Entropy of this k_epoch: 0.10478749871253967
Average policy_loss of this k_epoch: 0.0025076791644096375
KL Divergence Average Loss: 0.016095150262117386
Total Loss of this k_epoch: -0.0025707443710416555

===========================  [(62/64)]  ==========================

Entropy of this k_epoch: 0.10463745146989822
Average policy_loss of this k_epoch: 0.0024969354271888733
KL Divergence Average Loss: 0.01594918593764305
Total Loss of this k_epoch: -0.002575445454567671

===========================  [(63/64)]  ==========================



Epoch 32/32 (Inner K-Epochs): 100%|██████████| 64/64 [00:05<00:00, 12.41it/s]


Entropy of this k_epoch: 0.10390985757112503
Average policy_loss of this k_epoch: 0.0024606361985206604
KL Divergence Average Loss: 0.015799297019839287
Total Loss of this k_epoch: -0.0025768636260181665

===========================  [(64/64)]  ==========================

Entropy of this k_epoch: 0.10383570194244385
Average policy_loss of this k_epoch: 0.0024707019329071045
KL Divergence Average Loss: 0.01583714969456196
Total Loss of this k_epoch: -0.0025627119466662407

=====================  [Epoch (32/32)]  =====================
Last k_epoch stats:
Loss: -0.0025627 | Ratio: 0.9802344 | Entropy Term: 0.1038357


>>>>>>>>>>>>>>>>>>>>>
Main Epoch (Outer Loop): 100%|██████████| 32/32 [02:53<00:00,  5.42s/it]

===================  [Epoch (32/32)]  ===================
Entire Validation Dataset Accuracy: 0.9271| 178.0 / 192.0 samples
Training complete.


In [103]:
SAVE_LOCATION = "./models/trained_spam_model.pth"   # Define the model path and name of the trained model weights


torch.save(trained_policy.state_dict(), f=SAVE_LOCATION)

In [110]:
test_dataset = SpamDataset(csv_file="./sms_spam_collection/data_splits/test.csv", tokenizer=tokenizer)
test_dataloader = DataLoader(dataset=test_dataset, batch_size=args["dataloader_batch_size"], num_workers=args["dataloader_num_workers"], pin_memory=args["dataloader_pin_memory"], drop_last=True)


In [111]:
accuracy = evaluate_policy(trained_policy, test_dataloader, device='cuda')

Entire Dataset Accuracy: 0.9115 | 175.0 / 192.0 samples


In [112]:
input_text1 = "Hey, wanna go out to watch the new fantastic four movie?"
input_text2 = "XMAS Prize draws! We are trying to contact U. Todays draw shows that you have won a £2000 prize GUARANTEED. Call 09058094565 from land line. Valid 12hrs only"

In [ ]:
def simple_spam_classify_single(Policy, input_text, tokenizer, device='cpu'):
    Policy.eval().to(device)
    Softmax_lyr = torch.nn.Softmax(dim=-1)

    tokenized_text = tokenizer.encode(input_text)
    torch_text=torch.tensor(tokenized_text).unsqueeze(0)    # turn into a tensor and add a batch dimension
    model_inputs = torch_text.to(device)
    # print(f"torch_text: {torch_text} | {torch_text.shape}")
    with torch.no_grad():
        logits = Policy(model_inputs)[:,-1,:]
        Class_probabilities = Softmax_lyr(logits)
    prediction = torch.argmax(input=Class_probabilities, dim=-1)
    # print(f"prediction: {prediction}")

    print("==================================================================")
    print(f"Classifiying the following text:")
    print(f"[SPAM || NOT SPAM]: \n'{input_text}'")
    print(f"Prediction ... [ => {'SPAM' if prediction.item() == 1 else 'NOT SPAM'} <= ]")
    print("==================================================================")
    


In [ ]:
def simple_spam_classify_batch(Policy, input_text, tokenizer, device='cpu'):
    Policy.eval().to(device)
    Softmax_lyr = torch.nn.Softmax(dim=-1)

    tokenized_text = [
            tokenizer.encode(text) for text in input_text    # For each row in the text section of the pandas data frame tokenize the text string(sentence); creates list of token IDs for each example/item of the text data
        ]
    

    max_length = 0
    for encoded_text in tokenized_text:
        encoded_length = len(encoded_text)
        if encoded_length > max_length:
            max_length = encoded_length
    torch_text=torch.tensor(tokenized_text)    # turn into a tensor and add a batch dimension
    model_inputs = torch_text.to(device)
    # print(f"torch_text: {torch_text} | {torch_text.shape}")
    with torch.no_grad():
        logits = Policy(model_inputs)[:,-1,:]
        Class_probabilities = Softmax_lyr(logits)
    predictions = torch.argmax(input=Class_probabilities, dim=-1)
    print(f"predictions: {predictions}")

    bundle = zip(input_text, predictions.item())

    for i, (text_str, pred) in enumerate(bundle):
        print("==================================================================")
        print(f"Classifiying the following text:")
        print(f"[SPAM || NOT SPAM]: \n'{text_str}'")
        print(f"Prediction ... [ => {'SPAM' if pred == 1 else 'NOT SPAM'} <= ]")
        print("==================================================================")

In [129]:
device = "cuda" if torch.cuda.is_available() else "cpu"


In [ ]:
text_batch = [input_text1, input_text2]

In [ ]:
simple_spam_classify_batch(Policy=trained_policy, input_text=text_batch, tokenizer=tokenizer, device=device)


In [ ]:
simple_spam_classify_single(Policy=trained_policy, input_text=input_text1, tokenizer=tokenizer, device=device)

torch_text: tensor([[10814,    11, 18869,   467,   503,   284,  2342,   262,   649,  9623,
          1440,  3807,    30]]) | torch.Size([1, 13])
prediction: tensor([0], device='cuda:0')
Classifiying the following text:
[SPAM || NOT SPAM]: 
'Hey, wanna go out to watch the new fantastic four movie?'
Prediction ... [ => NOT SPAM <= ]


In [ ]:
def main(args):
    print("Setting up for Training")
    
    if args.device:     # Check if the user specified to use a CPU or GPU for training
        device = args.device
    else:
        if args.use_cuda:   # Check if the user wanted to use CUDA if available.
            device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


    BASE_CONFIG = {
        "vocab_size": 50257,     # Vocabulary size
        "context_length": 1024,  # Context length
        "drop_rate": 0.1,        # Dropout rate
        "qkv_bias": True         # Query-key-value bias
    }

    # Transfer to argparser setup
    gpt_size="gpt2-small (124M)"
    dataloader_batch_size=64
    num_workers=0
    pin_memory=True
    tokenizer=tiktoken.get_encoding("gpt2")

    print("Creating Datasets using train, test, and validation files.")

    prepare_datasets(data_file_path="./sms_spam_collection/SMSSpamCollection.tsv", store_directory="./sms_spam_collection/data_splits")

    train_dataset = SpamDataset(csv_file="./sms_spam_collection/data_splits/train.csv", tokenizer=tokenizer)
    test_dataset = SpamDataset(csv_file="./sms_spam_collection/data_splits/test.csv", tokenizer=tokenizer)
    validation_dataset = SpamDataset(csv_file="./sms_spam_collection/data_splits/validation.csv", tokenizer=tokenizer)
    
    train_dataloader = DataLoader(dataset=train_dataset, batch_size=dataloader_batch_size, num_workers=num_workers, pin_memory=pin_memory, drop_last=True)
    test_dataloader = DataLoader(dataset=test_dataset, batch_size=dataloader_batch_size, num_workers=num_workers, pin_memory=pin_memory, drop_last=True)
    validation_dataloader = DataLoader(dataset=validation_dataset, batch_size=dataloader_batch_size, num_workers=num_workers, pin_memory=pin_memory, drop_last=True)

    print("Beginning Training Script")
    start_time=time.time()

    trained_policy = grpo_train(
        model_config=BASE_CONFIG,
        train_dataloader=train_dataloader,
        validation_dataloader=validation_dataloader,
        gpt_size=gpt_size,
        epochs=args.epochs,
        learning_rate=args.learning_rate,
        batch_size=args.batch_size, # Significantly larger batch size recommended for stability
        k_epochs=args.k_epochs,
        epsilon=args.epsilon,
        beta_kl=args.beta_kl,
        entropy_coeff=args.entropy_coeff,
        log_iterations=args.log_iterations,
        gamma=args.gamma,
        device=device,
        num_envs=args.num_envs
    )
    end_time=time.time()

    elapsed_time= end_time - start_time
    hrs = int(elapsed_time / 3600)
    min = int((elapsed_time % 3600) / 60)
    seconds_remaining = elapsed_time - (hrs * 3600 ) - (min * 60)
    print(f"FINISHED MODEL TRAINING. \nTRAINING TOOK: {hrs} Hours, {min} Minutes, and {seconds_remaining} Seconds")


    print("\nTesting the trained policy:")

    test_dataset_accuracy = evaluate_policy(trained_policy, test_dataloader, current_epoch=None, max_epochs=None, device=device)

 #---------------  !!!  ---------------
    SAVE_LOCATION = "./model/trained_model.pth"   # Define the model path and name of the trained model weights

    if args.save_model:     # Check if the user wants to save the trained model weights
        if args.model_output_path:     # Check if the user specified a target save location
            SAVE_LOCATION=args.model_output_path
        
        torch.save(trained_policy.parameters(), f=SAVE_LOCATION)
        print(f"Model weights saved in: {SAVE_LOCATION}")

    print("Finished Running Script")

In [ ]:
# Example usage (assuming you have a way to call this function, e.g., in a main block)
if __name__ == '__main__':
    parser = argparse.ArgumentParser(description="Train and test a BlackJack PPO agent.")

    # Add arguments
    parser.add_argument('--epochs', type=int, default=2000,
                        help='Number of training epochs.')
    parser.add_argument('--learning_rate', type=float, default=0.0003,
                        help='Learning rate for the optimizer.')
    parser.add_argument('--dataloader_batch_size', type=int, default=64,
                        help='Dataloader Batch sizes for train, test, validation data files.')
    parser.add_argument('--batch_size', type=int, default=1024,
                        help='Batch size for training.')
    parser.add_argument('--gpt2_size', type=str, default="gpt2-small (124M)",
                        help='GPT2 size for model construction.')
    parser.add_argument('--k_epochs', type=int, default=128,
                        help='Number of policy update epochs per trajectory collection.')
    parser.add_argument('--epsilon', type=float, default=0.2,
                        help='Clipping parameter for PPO.')
    parser.add_argument('--beta_kl', type=float, default=0.01,
                        help='KL divergence coefficient (for PPO-like algorithms).')
    parser.add_argument('--entropy_coeff', type=float, default=0.001,
                        help='Entropy regularization coefficient.')
    parser.add_argument('--log_iterations', type=int, default=100,
                        help='Log training progress every N iterations.')
    parser.add_argument('--gamma', type=float, default=0.99,
                        help='Discount factor for rewards.')
    parser.add_argument('--num_envs', type=int, default=16,
                        help='Number of parallel environments for training.')
    parser.add_argument('--use_cuda', action='store_true',
                        help='Use CUDA if available.')
    parser.add_argument('--device', type=str, default='cpu',
                        help='Explicitly set device (e.g., "cpu, cuda:0", "cpu"). Overrides --use_cuda if specified.')
    parser.add_argument('--save_model', action='store_true',
                        help='Save the trained model weights.')
    parser.add_argument('--model_output_path', type=str, default='blackjack_policy_model.pth',
                        help='Path to save the trained model weights.')

    # Parse the arguments
    args = parser.parse_args()

    
    main(args)